In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

# List collections
print(client.get_collections())

# Check info collection
info = client.get_collection("chat_history")
print(info)


collections=[CollectionDescription(name='rag_knowledge'), CollectionDescription(name='chat_history')]
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=2 segments_count=6 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_thr

In [5]:
from qdrant_client import QdrantClient
from pprint import pprint

client = QdrantClient(host="localhost", port=6333)

def inspect_qdrant():
    collections = client.get_collections().collections
    print(f"Found {len(collections)} collections:")
    
    for col in collections:
        print("\n" + "="*40)
        print(f"Collection name: {col.name}")
        try:
            info = client.get_collection(col.name)
            print(f"Points count: {info.points_count}")
            vectors = info.config.params.vectors
            if isinstance(vectors, dict):
                print("Vector names & sizes:")
                for name, vcfg in vectors.items():
                    print(f"  - {name}: size={vcfg.size}, distance={vcfg.distance}")
            else:
                print(f"Vector config: size={vectors.size}, distance={vectors.distance}")
        except Exception as e:
            print("Could not fetch collection info:", e)
        
        print("Sample points (up to 10):")
        try:
            resp, _ = client.scroll(col.name, limit=10, with_payload=True)
            points = getattr(resp, "result", resp)
            for p in points:
                vector_names = list(p.vector.keys()) if isinstance(p.vector, dict) else ["default"]
                print(f"  id={p.id}, vector_names={vector_names}, payload={p.payload}")
        except Exception as e:
            print("Could not fetch points:", e)

if __name__ == "__main__":
    inspect_qdrant()


Found 1 collections:

Collection name: collection
Points count: 0
Vector config: size=384, distance=Cosine
Sample points (up to 10):


In [6]:
from qdrant_client import QdrantClient

# Kết nối Qdrant (ví dụ chạy local)
client = QdrantClient("http://localhost:6333")

collection_name = "collection"

# Lấy thông tin collection
info = client.get_collection(collection_name)
print("Tổng số vector:", info.vectors_count)


Tổng số vector: None


In [7]:
import numpy as np

# Giả sử bạn có embedding vector test 
dummy_vector = np.random.rand(384).tolist()

result = client.search(
    collection_name=collection_name,
    query_vector=dummy_vector,
    limit=3
)

print("Kết quả tìm được:", result)


Kết quả tìm được: []


C:\Users\admin\AppData\Local\Temp\ipykernel_14208\2745754574.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  result = client.search(


In [10]:
import qdrant_client

def check_collections():
    client = qdrant_client.QdrantClient(host="localhost", port=6333)

    collections = client.get_collections()
    print("📂 Danh sách collections trong Qdrant:")
    for coll in collections.collections:
        name = coll.name

        # Dùng count để chắc chắn có số vector
        count = client.count(collection_name=name, exact=True).count
        print(f" - {name}: {count} vectors")

    if not collections.collections:
        print("⚠️ Không có collection nào trong Qdrant (rỗng).")

if __name__ == "__main__":
    check_collections()


📂 Danh sách collections trong Qdrant:
 - collection: 1 vectors


In [5]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")

# Xem các collection hiện có
print(client.get_collections())

# Nếu có collection "documents", xem thử số lượng points
print(client.count(collection_name="collection", exact=True))

# Lấy vài point mẫu
points = client.scroll(collection_name="collection", limit=5)
print(points)


collections=[CollectionDescription(name='collection')]
count=0
([], None)


In [14]:
# Test case
from modules.core.state import GlobalState

state = GlobalState(user_input="xin chào")
processed_state = processor_instance.process_query(state)

print(f"Skip retrieval: {processed_state.skip_retrieval}")  # True
print(f"Final answer: {processed_state.final_answer}")      # "Xin chào! Tôi có thể giúp gì cho bạn?"

TypeError: GlobalState.__init__() got an unexpected keyword argument 'user_input'